## Sample script for AI moderator

### Setup - atproto

- langchain           0.1.11
- langchain-openai    0.0.8

In [ ]:
!pip install -U git+https://github.com/FujitsuResearch/atproto-python.git
!pip install -U langchain langchain-openai

In [ ]:
!git clone https://github.com/FujitsuResearch/sample-moderator.git
%cd sample-moderator/

In [ ]:
from atproto import Client

handle = "<your handle>"
password = "<your password>"
channel = "<Your channel>"
api_url = "https://federated-sns-server[xx].research.global.fujitsu.com/xrpc"
token = "<Your Azure AD token>"
prompt_file = "input/test-prompt1.txt" # sample prompt file
client = Client(base_url=api_url)

### Setup - langchain

In [ ]:
# Docker python:3.12-bookworm for OpenAI
import os 
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

os.environ["OPENAI_API_KEY"] = "<set OpenAI api key>"

chat = ChatOpenAI(model_name="gpt-3.5-turbo",
                  max_tokens=300)

### Get posts and run LLM

In [ ]:
# Read a prompt
from langchain.prompts import PromptTemplate

with open(prompt_file) as f:
    template = f.read()
    print(template)

prompt = PromptTemplate(
    template=template,
    input_variables = ["post"],
)

In [ ]:
# Log-in
try:
    profile = client.login(handle, password, portal_token=token)
    print(f"Successfully logged in: {profile.handle}")
    
except Exception as e :
    print(e)

# List joined channels
response = client.app.fujitsu.channel.list_channel_info()
joined_channel_list = [cinfo.channel_handle for cinfo in response.channel_info if channel in cinfo.channel_handle ]

channel_name = joined_channel_list[0]
print("Selected channel: ", channel_name)

In [ ]:
limit = 5
flag = True
cursor = None
while flag :
    response = client.get_channel_records(
                    channel=channel_name,
                    cursor=cursor,
                    limit=limit,
                )
    if len(response.records) < limit :
        flag = False
    
    cursor = response.cursor
    
    for record in response.records :
        value = record.value.model_dump()
        handle = value['handle']
        text = value['text']
        
        formatted_prompt = prompt.format(post=text)
        messages = [HumanMessage(content=formatted_prompt)]
        result = chat.invoke(messages).content
        
        print(formatted_prompt)
        print(result)
        print("---")
